<a href="https://colab.research.google.com/github/ucfilho/RTC_case_02_optimization/blob/master/RTC_DE_case_02_aproach_future_mar_31_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from scipy.optimize import differential_evolution
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# Functions used in this code
# buildMatriz(x): return three slots (Ind_I,Ind_J,deltaT,frac, use_frac)
#                 the function also returns the Tout, Tion
#                 use_frac and frac represent 
#                 Ind_I,Ind_J represents streams (hot i and cold j)
#                 deltaT represents hot temperature changing
# Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,Tin,Tout):
#                 return temperatures of particles after gain or loss of energy
#                 return heat load of particles after gain or loss of energy
#                 return error to apply penalty (error=1 is penalty)
# Area(U,Tin,Tout,Ind_I,Ind_J,deltaT):
#                 return Area of heat exchanger used in heat and cold  streams
# Utilidades(tempAlvo,Tout,mCp):
#                 return Area of heat exchanger using utilities
#                 return the kW of utilities used
# CustoProcesso(Area,Utilidades,cargaUFtodas,cargaUQtodas):
#                 return the cost composed by use of utilities and depreciation 
#                 of heat exchangers
# FUN(x): return the cost
#                integrate the use of all functions described above
# Variables used in this code:
# x: population of particles
# NPAR: number of particles
# Trocas: number of exchanges allowed between the streams
# mCp=[4,2,1.5,3] #[kW/K]
# tempInicial initial temperature in each strem (kelvin)
# tempAlvo target of temperatures to be reached by each stream
# cost function to be optimized
# mCp heat capacity of stream (composed by flux multiplied by Cp)

In [3]:
!git clone https://github.com/ucfilho/raianars_RTC_optimization #clonar do Github
%cd raianars_RTC_optimization
import de_bounds_matriz

fatal: destination path 'raianars_RTC_optimization' already exists and is not an empty directory.
/content/raianars_RTC_optimization


In [0]:
def Shark_buildMatriz(x):
  global Trocas, tempInicial, tempAtual
  # nova versao com divisao da corrente --segue a logica
  #         Ind_I,Ind_J,deltaT, use_frac,frac=Shark_buildMatriz(x)
  # use_frac zero nao divide a corrente 1 divide a correntr
  # frac fracao da corrente que e gerada depois da divisao
  # a variavel x tem 3*trocas+3 elementos (para trocas=4, x tem 15)
  NPAR=1
  cols=len(tempInicial)
  size=int(Trocas*3+3)
  x=np.array(x).reshape(1,size)
  # print('x=');print(x)
  Ind_I=np.zeros((NPAR,Trocas)).astype(np.int64)
  Ind_J=np.zeros((NPAR,Trocas)).astype(np.int64)
  #use_frac=np.zeros((NPAR,Trocas))
  #frac=np.zeros((NPAR,Trocas))
  deltaT=np.zeros((NPAR,Trocas))

  for i in range(NPAR):
    for j in range(Trocas):
      #print('[i,j]=%d %d'%(i,j))
      Ind_I[i,j]=round(x[i,j])
      Ind_J[i,j]=round(x[i,j+Trocas])
      deltaT[i,j]=x[i,j+2*Trocas]


  frac=x[i,3*Trocas]
  if(x[i,1+3*Trocas]<0.5):
    use_frac= 0
  else:
    use_frac= 1.0
  Split=int(abs(x[i,2+3*Trocas]-0.5)) # o valor 0.5 negativo : mesma prob
  return Ind_I,Ind_J,deltaT, use_frac,frac,Split

In [0]:
def buildMatriz(x):
  global Trocas, tempInicial, tempAtual 
  NPAR=1
  cols=len(tempInicial)
  size=3*cols+3 # three additional positions representing split (which,value, yes or not)
  x=np.array(x).reshape(1,size)
  # print('x=');print(x)
  Ind_I=np.zeros((NPAR,Trocas)).astype(np.int64)
  Ind_J=np.zeros((NPAR,Trocas)).astype(np.int64)
  Tin=np.zeros((NPAR,Trocas))
  Tout=np.zeros((NPAR,Trocas))
  deltaT=np.zeros((NPAR,Trocas))
  for i in range(NPAR):
    for j in range(Trocas):
      #print('[i,j]=%d %d'%(i,j))
      Ind_I[i,j]=round(x[i,j])
      Ind_J[i,j]=round(x[i,j+Trocas])
      deltaT[i,j]=x[i,j+2*Trocas] 
    for jj in range(cols):
      Tin[i,jj]=tempInicial[jj]
      Tout[i,jj]=tempAtual[jj]
  return Ind_I,Ind_J,deltaT, Tin, Tout

In [0]:
def Temperatura(x):
  global Trocas, mCp,tempInicial
  
  NPAR=1
  #Ind_I,Ind_J,deltaT,Tin, Tout=buildMatriz(x) 
  Ind_I,Ind_J,deltaT, use_frac,frac,Split=Shark_buildMatriz(x)
  Tatual=np.copy(np.array(tempInicial))
  Tout=np.zeros((NPAR,(Trocas+1)))

  for ki in range(NPAR):
    cont=-1
    for kj in range(Trocas):
      cont=cont+1 
      if( kj==Split): # two streams has the same tempInicial (split's rule)
        Tout[ki,cont]=tempInicial[kj]
        cont=cont+1
        Tout[ki,cont]=tempInicial[kj]
      else:
        Tout[ki,cont]=tempInicial[kj]

  erro=np.zeros((NPAR)) # when erro=1 there are penalties
  cont=-1
  for ki in range(NPAR):
    for kj in range(Trocas): # add split to one stream Trocas+1 but dont work 
      cont=cont+1
      k=kj
      i=Ind_I[ki,kj]
      j=Ind_J[ki,kj]
      
      tempSaidaQ=Tout[ki,i]+deltaT[ki,kj]
      tempSaidaF=-mCp[kj]*deltaT[ki,kj]/(mCp[j])+Tout[ki,j]
      flag=0 # flag : indica que nao entra no teste logico
      # usei flag no lugar q ifs pq a identacao ficou melhor assim
      if(Tout[ki,i] > Tout[ki,j]): #Tquente maior Tfria: vaores atuais
        if(tempSaidaQ > tempSaidaF): #Tquente maior Tfria: saidas
          flag=1
      if(flag>0):
        Tout[ki,i]=tempSaidaQ
        Tout[ki,j]=tempSaidaF
      else:
        erro[ki]=1

  return Tout,erro

In [0]:
def Area(x):

  global mCp,tempAlvo,tempInicial,Trocas,U
  
  NPAR=1
  
  Area=[]

  #Ind_I,Ind_J,deltaT,Tin, Tout=buildMatriz(x) 
  Ind_I,Ind_J,deltaT, use_frac,frac,Split=Shark_buildMatriz(x)
  Tatual=np.copy(np.array(tempInicial))

  rows=Trocas+1 # add split to one stream 
  cols=len(mCp)

  Area_Particula=[]

  for k in range(Trocas+1): # add split to one stream 

    i=Ind_I[0,k]
    j=Ind_J[0,k]
    tempSaidaQ=Tatual[i]+deltaT[0,k]
    tempSaidaF=-deltaT[0,k]*mCp[i]/(mCp[j])+Tatual[j]

    Told_i=Tatual[i]
    Told_j=Tatual[j]

    Tatual[i]=np.copy(tempSaidaQ)
    Tatual[j]=np.copy(tempSaidaF)
    deltaT1=Tatual[i]-tempSaidaF
    deltaT2=tempSaidaQ-Tatual[j]
    try:
      A=(deltaT1-deltaT2)
      B=np.log((deltaT1/deltaT2))
      deltaTMediaLog=A/B
    except:
      deltaTMediaLog=1e99
  
    if(deltaTMediaLog==1e99):
      AreaCalc=1e99
    else:
      AreaCalc=abs(deltaT[0,k]*mCp[i]/(U*deltaTMediaLog))

    if(Told_i==Tatual[i]):
      AreaCalc=0

    if(Told_j==Tatual[j]):
      AreaCalc=0

      Area_Particula.append(AreaCalc)
      
  Area.append(Area_Particula)

  return Area

In [0]:
def Utilidades(tempAlvo,Tout,mCp):

  cols=len(mCp)
  NPAR=1

  
  '''
  caso I cita TUQin=553 e TQout=552, TUFin=303 TUFout=353
  '''
  
  TUQin=553
  TUQout=552
  TUFin=303 
  TUFout=353
  
  Areas=[]
  cargaUQtodas=[]
  cargaUFtodas=[]

  for ki in range(NPAR):
    areaTrocador=[]
    cargaUQ=0
    cargaUF=0
    for j in range(cols):
      delta=tempAlvo[j]-Tout[ki,j]
      if(delta>0):#usa UQ
        cargaUQ=cargaUQ+mCp[j]*delta
        deltaT1=(TUQin-tempAlvo[j])
        deltaT2=(TUQout-Tout[ki,j])

        try:
          A=deltaT1-deltaT2
          B=np.log(deltaT1/deltaT2)
          deltaTMediaLog=A/B
        except:
          deltaTMediaLog=1e99

        if(deltaTMediaLog==1e99):
          areaTrocador.append(1e99)
        else:
          areaTrocador.append((abs(mCp[j]*(delta)/(U*deltaTMediaLog))))

      elif(delta==0):#nao usa UF e UQ
        deltaTMediaLog=0
        areaTrocador.append(0) 

      else:#usa UF
        deltaT1=(Tout[ki,j]-TUFout)
        deltaT2=(tempAlvo[j]-TUFin)     
        cargaUF=cargaUF+mCp[j]*(-delta)

        #UF -> água resfriada -> entra a 303K e sai a 353K
        try:
          A=deltaT1-deltaT2
          B=np.log(deltaT1/deltaT2)
          deltaTMediaLog=A/B
        except:
          deltaTMediaLog=1e99
        
        if(deltaTMediaLog==1e99):
          areaTrocador.append(1e99)
        else:
          areaTrocador.append((abs(mCp[j]*(-delta)/(U*deltaTMediaLog))))

    cargaUQtodas.append(cargaUQ)
    cargaUFtodas.append(cargaUF)

    Areas.append(areaTrocador)

  return Areas, cargaUQtodas,cargaUFtodas 


In [0]:
def CustoProcesso(Area,Utilidades,cargaUFtodas,cargaUQtodas):
  global coefCustoUQ,coefCustoUF
  
  NPAR=1

  cost=np.zeros((NPAR))
  for i in range(NPAR):
    cargaUQ=cargaUQtodas[i]
    cargaUF=cargaUFtodas[i]
    areaTrocadores=Area[i]
    areaUtilidades=Utilidades[i]
    costTrocadores=0
    for it in areaTrocadores:
      for wii in areaUtilidades:
        if(it<1e99):
          if(wii<1e99):
            costTrocadores=costTrocadores+300*(it**0.5+wii**0.5)
          else:
            costTrocadores=1e99
            break
    costUQ=coefCustoUQ*cargaUQ
    costUF=coefCustoUF*cargaUF
    
    if(costTrocadores==1e99):
      cost[i]=1e99
    else:
      cost[i]=costUQ+costUF+costTrocadores
      
  return cost

In [0]:
def Numb(x):
  global mCp,tempAlvo,tempInicial,Trocas

  Ind_I,Ind_J,deltaT,Tin, Tout=buildMatriz(x) # transforma x em matriz logica e troca
  Tatual=np.copy(np.array(tempInicial))

  rows=Trocas 
  cols=len(mCp)
  Ttrocas=np.zeros((rows+4,cols))# feito em quaduplicata so para melhorar o visual
  for i in range(2):
    for j in range(cols):
      Ttrocas[i,j]=np.copy(Tatual[j])
  cont=1
  #Ttrocas.append(Tatual)
  #Ttrocas.append(Tatual) # feito em duplicata so para melhorar o visual
  for k in range(Trocas):
    cont=cont+1
    i=Ind_I[0,k]
    j=Ind_J[0,k]
    tempSaidaQ=Tatual[i]+deltaT[0,k]
    tempSaidaF=-deltaT[0,k]*mCp[i]/(mCp[j])+Tatual[j]
    Tatual[i]=np.copy(tempSaidaQ)
    Tatual[j]=np.copy(tempSaidaF)

    for jj in range(cols):
      Ttrocas[cont,jj]=Tatual[jj]  


  for jj in range(cols): # feito em duplicata so para melhorar o visual
    Ttrocas[cont+1,jj]=tempAlvo[jj]
    Ttrocas[cont+2,jj]=tempAlvo[jj]

  
  return Ttrocas


In [0]:
#FUN calcula o TAC para um determinado arranjo
#  cada calculo é feito em uma funcao a parte Fun usa todas funcoes 

def FUN(x):

  global mCp,tempAtual,tempAlvo,tempInicial,U,coefCustoUQ,coefCustoUF
  
  NPAR=1
  
  funr=np.zeros((NPAR))
  cols=len(mCp)
  Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(x) # transforma x em matriz logica e troca
  Tout, Erro=Temperatura(x)
  areaTrocadores =Area(x)
  areaUtilidade, cargaUQ,cargaUF = Utilidades(tempAlvo,Tout,mCp)
  cost=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
  funr=cost

  for k in range(NPAR):
    if(Erro[k]>0):
      funr[k]=1e99
  funr = np.where(np.isnan(cost), 1e99, cost)
  
  # print(funr)

  return funr 

In [0]:
#********************PROGRAMA PRINCIPAL*************************

Trocas=4  # number of heat exchanges between the streams (not utilities)
# Trocas=Trocas+1 # addition of split in one of the streams
#                 # but not here (inside the routines)
mCp=[4,2,1.5,3] #[kW/K]
tempInicial=[453.0,393.0,523.0,533.0] #[K]
tempAtual=np.copy(tempInicial) 
#tempAtual=[453,393,523,533] #[K]
tempAlvo=[513.0,508.0,403.0,433.0] #[K]
U=0.2 #[kW/(m2.K)]
coefCustoUQ=110 #[$/(kW.ano)]
coefCustoUF=12.2 #[$/(kW.ano)]

# Trocas=Trocas+1




#MAX=np.array([3,3,3,3,1,1,1,1,0,0,0,0])
#MIN=np.array([2,2,2,2,0,0,0,0,-100,-100,-100,-100])
MAX=np.array([3,3,3,3,3,3,3,3,0,0,0,0])
MIN=np.array([0,0,0,0,0,0,0,0,-100,-100,-100,-100])
#MAX=np.array([3,2,3,2,1,1,1,1,0,0,0,0])
#MIN=np.array([3,2,3,2,1,1,1,1,-100,-100,-100,-100])
#MAX=np.array([3,3,3,3,1,1,1,1,0,0,0,0])
#MIN=np.array([2,2,2,2,0,0,0,0,-100,-100,-100,-100])
#MIN=np.array([2,2,2,2,0,0,0,0,-40,-40,-40,-40])





bounds=[]
for k in range(len(MAX)):
  bounds.append((MIN[k],MAX[k]))


# print(bounds)



In [13]:
fobj=FUN
mut=0.5
crossp=0.5
MAX=np.array([3,3,3,3,3,3,3,3,0,0,0,0,1,1,Trocas])
MIN=np.array([0,0,0,0,0,0,0,0,-100,-100,-100,-100,0,0,0])
cols=15
NPAR=10
its=20

x=np.zeros((NPAR,cols))
for i in range(NPAR):
  for j in range(cols):
    r=np.random.random()
    x[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

x,BEST,FOBEST,XY,BEST_XY=de_bounds_matriz.de(bounds, mut, crossp, 
                                             NPAR, its,fobj,x)

NameError: ignored

In [14]:
print('FOBEST=');print(FOBEST)

FOBEST=


NameError: ignored

In [15]:
print(BEST)

NameError: ignored

In [16]:
Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(BEST)
print(Ind_I)
print(Ind_J)
print(deltaT)


NameError: ignored

In [0]:
Busca=Numb(BEST)
print(pd.DataFrame(Busca))
df=pd.DataFrame(Busca)

In [17]:
print(df)


NameError: ignored

In [18]:

Streams=4
Change=range(Streams+4) # Streams+4 so para ficar com um visual melhor
for k in range(Streams):
  plt.plot(Change,df.iloc[:,k])
plt.xlabel('Stages')
plt.ylabel('Temperature (K)')


NameError: ignored

In [19]:
print(tempInicial)
print(tempAlvo)
print(df)

[453.0, 393.0, 523.0, 533.0]
[513.0, 508.0, 403.0, 433.0]


NameError: ignored

In [0]:
# Trocas=4 assim varia de 4 em 4 exceto os dois ultimos
#  Os quatro primeiros representa o indice que perde calor
#  Os quatro seguintes representa a corrente que aquece
#  Os quatro seguintes representa a diferenca de temperatura que resfria
#  O antepenultimo tem use_frac=0 quando nao troca e use_frac=1.0 na troca
#  O penultimo representa a fracao de troca mas so troca se use_frac=1.0
#  O ultimo representa em qual troca tera a divisao da corrente Stream
MAX=np.array([3,3,3,3,3,3,3,3,0,0,0,0,1,1,Trocas])
MIN=np.array([0,0,0,0,0,0,0,0,-100,-100,-100,-100,0,0,0])
NPAR=1
cols=15
w=np.zeros((NPAR,cols))
for i in range(NPAR):
  for j in range(cols):
    r=np.random.random()
    w[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
Ind_I,Ind_J,deltaT, use_frac,frac,Split=Shark_buildMatriz(w)
print(Ind_I)
print(Ind_J)
print(deltaT)
print(use_frac) # this varible zero or one (type real) describes 
                # avoid split when represents zero value
print(frac) # 'frac' and '1-frac' represent the amout of strem after the split
print(Split) # this varible (type int describes which stream is split)